In [3]:
from langchain_community.document_loaders import TextLoader

loaders = TextLoader("speech.txt")

In [4]:
text_documents = loaders.load()
text_documents[0]

Document(metadata={'source': 'speech.txt'}, page_content='Good morning everyone.\n\nIn today’s digital world, data plays a crucial role in almost every decision we make. From recommending the next video we watch to helping doctors diagnose diseases, data drives intelligence across industries.\n\nHowever, data by itself has no value unless it is properly collected, cleaned, and understood. Raw data often contains noise, inconsistencies, and missing information. Effective data ingestion ensures that information from multiple sources is transformed into a usable and reliable format.\n\nAs technology continues to evolve, the ability to ingest and process data efficiently will become even more important. Those who understand how to work with data will be better equipped to build intelligent systems and make informed decisions.\n\nThank you.')

In [5]:
print(len(text_documents))

1


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 5


True

In [7]:
from langchain_community.document_loaders import WebBaseLoader
import bs4


loader2= WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                           class_= ("post-title","post-content","post-header")
                       )),)


text_documents2 = loader2.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader3= PyPDFLoader("attention.pdf")

In [9]:
text_documents3 = loader3.load()

In [10]:
text_documents3[0]


Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszka

In [11]:
print(type(text_documents3))

<class 'list'>


In [16]:
# here we are doing chunking of the documents
import langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(text_documents3)

In [18]:
print(len(docs))

52


In [24]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 970aa74c0a90:   0% ▕                  ▏ 129 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   0% ▕                  ▏ 235 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   0% ▕                  ▏ 403 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   0% ▕                  ▏ 469 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   1% ▕                  ▏ 1.4 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   1% ▕                  ▏ 1.5 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:   1% ▕                  ▏ 1.7 MB/274

In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [25]:
db = Chroma.from_documents(docs, OllamaEmbeddings(model="nomic-embed-text"))

In [22]:
!pip show langchain-community

Name: langchain-community
Version: 0.4.1
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/rag/lib/python3.10/site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain-classic, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [26]:
query = " what is the attention function?"
result = db.similarity_search(query,k = 3)

In [28]:
print(result[0])

page_content='3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
3' metadata={'trapped': '/False', 'creationdate': '2024-04-10T21:11:43+00:00', 'subject': '', 'title': '', 'source': 'attention.pdf', 'producer': 'pdfTeX-1.40.25', 'moddate': '2024-04-10T21:11:43+00:00', 'page': 2, 'creator': 'LaTeX with hyperref', 'keywords': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'total_pages': 15, 'author': '', 'page_label': '3'}


In [29]:
## home work use the langs vector database